In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from numpy.f2py.auxfuncs import throw_error

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import euclidean_distances
from scipy.spatial.distance import cdist

import warnings



### Création de l'authentification API Spotify

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

SPOTIFY_CLIENT_ID = "ed9ece4a286d42f1aa834c82bed1f95e"
SPOTIFY_CLIENT_SECRET = "e1f250bb36914f3c9235c88df325eff6"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=SPOTIFY_CLIENT_ID,
    client_secret=SPOTIFY_CLIENT_SECRET,
    redirect_uri="http://localhost:8000/callback",
    scope="user-top-read playlist-read-private user-library-read playlist-modify-public playlist-modify-private",
    cache_path="./spotify_cache",
    state="unique_state_value"
))

In [3]:
import os

if os.path.exists("./spotify_cache"):
    os.remove("./spotify_cache")
    print("Custom cache file deleted.")
else:
    print("Custom cache file not found.")

Custom cache file deleted.


### Get the user's top tracks

In [4]:
top_tracks = sp.current_user_top_tracks(limit=50, time_range='long_term')

print("Your top tracks:")

for idx, track in enumerate(top_tracks['items']):
    print(f"{idx + 1}: {track['name']} by {track['artists'][0]['name']}")
    

Your top tracks:
1: Numb by Linkin Park
2: Comme c'est bon by Jenifer
3: Wakfu opening song by Guillaume Houzé
4: Stopped Time by WOO JI HUN
5: Iron Heart by Marina Kaye
6: Descendants Of The Sun (Original Television Soundtrack) Pt.6 - Talk Love by K.Will
7: Mauvais rêve by Hoshi
8: One Piece (1999) - The Very, Very, Very Strongest by Geek Music
9: My Sails Are Set (from the Netflix Series "One Piece") by Sonya Belousova
10: Believe in Myself (Fairy Tail Opening 21) by PelleK
11: Black Rover by VK Blanka
12: Time of JungHyuk for Seri by Nam Hye Seung
13: Dépassé by Nuit Incolore
14: Un diavolo scaccia l′altro by Lee Nyeom
15: ワノ国 幕開け、幕引きのテーマ by Kohei Tanaka
16: 悲壮 by Yasuharu Takanashi
17: You Give Love A Bad Name by Bon Jovi
18: Popcorn salé - Live by Les Enfoirés
19: Someone You Loved by Lewis Capaldi
20: 疾風組曲 by Yasuharu Takanashi
21: STRIKE BACK by BACK-ON
22: Across the Stars (Love Theme from "Star Wars: Attack of the Clones") by John Williams
23: Black Catcher by VK Blanka
24: Sc

### Get the track IDs to fetch audio features

In [5]:
track_ids = [track['id'] for track in top_tracks['items']]

print(f"Track IDs: {track_ids}")

Track IDs: ['2nLtzopw4rPReszdYBJU6h', '2342PpFWk3V3oUsFmfOzsI', '550268c5oupgwaenURSsE9', '1pRapazgWNxpEqTqe63jH5', '4T0cHbcxGzEGzUki89NFp9', '5NkpsRKvSBukdYQtl3WLZW', '36HkKVTxGk1jIoUqW0AnUi', '4O4u9XSr2vgZxLLNMOq68V', '507tVcKoaL0iORuaM2nrmy', '4Ik2HBEoqvF3gIOYnMHRM6', '04LCCxG3pbOYeENiyUc7Xb', '0IjT3ZixP581jP70WQ3nbv', '5bOWPM362XpMa8zn6UR9te', '7AnHS1o08tFXLmnOedcucv', '15WLsibdUCBTyN6a9YUajB', '6JeKu6FAAmfGrr2tTBnVIF', '0rmGAIH9LNJewFw7nKzZnc', '5sK0uWylEvnMPGBUatnZnl', '7qEHsqek33rTcFNT9PFqLf', '46tIwaDhLsDhrYl6vjK8ZB', '5QSt0jnVuPdQsC9plgW3lz', '757Azo0GkANYBAaNackdry', '5RXmLhscstsdvWPN0XtUa0', '5eSGz8BIIoUIUWPhIp5CWy', '7r7PL7TxcixHculcGtnEC9', '210bN1eoP343zkIqIv6mne', '7ojDz4j9rqmnLdjsfsU3nk', '7JoZiEMrwjpD18krCV8Czo', '2ph6BRhGcUzc2W9wIulG5k', '3di5hcvxxciiqwMH1jarhY', '3bNv3VuUOKgrf5hu3YcuRo', '6ZFbXIJkuI1dVNWvzJzown', '3uBgxxlZ3Ul8nH7NUdOlRZ', '5Do3qmFMpCjVLmtLhmj3a1', '6mqOtq4LgQ4eK6yF9yZvO7', '16NThvaQeF0xItfMI6XGwn', '22qLxnlZDOEP7LwSd0S2tr', '5DWKsSCWDUf9ffzvAlwInE', 

### Load data_appended dataset and fetch the top tracks using ids

In [6]:
data = pd.read_csv("data/data_appended.csv")

top_tracks_data = data[data['id'].isin(track_ids)]

print("Top tracks data :")
print("Found a total of {} tracks".format(len(top_tracks_data)))
print(top_tracks_data)

track_to_analyse = top_tracks_data['id'].tolist()

print(track_to_analyse)
print("Tracks that are not in top track :")
for idx,track in enumerate(top_tracks['items']):
    if track['id'] not in top_tracks_data['id'].values:
        print(f"{idx}: {track['name']} by {track['artists'][0]['name']}")

Top tracks data :
Found a total of 18 tracks
        valence  year  acousticness                   artists  danceability  \
12808    0.8120  1986      0.066300              ['Bon Jovi']        0.5410   
15106    0.6090  1997      0.064700     ['Natalie Imbruglia']        0.5750   
16206    0.2430  2003      0.004600           ['Linkin Park']        0.4960   
16609    0.3610  2005      0.269000              ['The Fray']        0.6400   
17740    0.0401  2010      0.155000           ['Hans Zimmer']        0.2210   
19413    0.4460  2019      0.751000         ['Lewis Capaldi']        0.5010   
72952    0.0382  2009      0.692000          ['James Horner']        0.0993   
89598    0.1750  2007      0.572000       ['Steve Jablonsky']        0.3920   
123630   0.5890  2014      0.005350               ['BACK-ON']        0.4690   
154974   0.5390  2018      0.002150           ['Vickeblanka']        0.5990   
170191   0.3540  2018      0.062000         ['Bishop Briggs']        0.5240   
174556 

In [7]:
#Throw an error to stop the execution here
throw_error()

TypeError: throw_error.__init__() missing 1 required positional argument: 'mess'

In [8]:
from useKeam import FindSimilarSong

f = FindSimilarSong()
result = f.find_songs_for_playlist(track_to_analyse)




### Create Spotify playlist

In [9]:
# Get user ID (required to create a playlist)
user_id = sp.current_user()['id']

# Create a new playlist
playlist_name = "AI-ify Playlist"
playlist_description = "This is a playlist created using AI-ify."
playlist = sp.user_playlist_create(user_id, playlist_name, public=True, description=playlist_description)

# Add tracks to the playlist
sp.playlist_add_items(playlist['id'], result)

{'snapshot_id': 'AAAAAr19XzDm0/shmOuKWqwhSweyZVSO'}